# Initialization

In [1]:
import os
import shutil
import numpy as np

# Run simple simulation
## Load module

In [2]:
import PyRAMSES

## Load simulator

In [3]:
# kp: 0.1 ~ 20.10 (#41)
# ki: 0.1 ~ 20.10 (#41)
# td: 0.01 ~ 0.21 (#11)

for td in np.arange(0.01, 0.02, 0.02):  # td: 0.010 sec
    td = "{0:.2f}".format(round(td,2))
    for kp in np.arange(5.6, 5.7, 0.5):  # kp: 5.6
        kp = "{0:.2f}".format(round(kp,2))
        for ki in np.arange(3.6, 5.7, 0.5):  # ki: 3.6~5.6
            ki = "{0:.2f}".format(round(ki,2))

            print("kp = " + str(kp))
            print("ki = " + str(ki))
            print("td = " + str(td))

            ram = PyRAMSES.sim()

            # Load saved test-case
            case = PyRAMSES.cfg('cmd.txt')

            # Add one observation more
            case.addRunObs('MS g2') # will plot in real-time the voltage on bus g1

            # Run simulation and pause at t=15 seconds
            start_time=15
     
            flag = 0
            try:
                ram.execSim(case,start_time)
            except: # this "try...except..." should not be appear if this error was removed
                flag = 1
                print("IMPORTANT TIPS: system will be quited and the file will be created WITHOUT control........")
                # Kill gnuplot
                os.system("TASKKILL /F /IM gnuplot.exe /T")
                print("kill successfully: non-control")

                # End simulation and exit
                try:
                    ram.endSim()
                    print("end simulation and exit successfully: non-control")
                except:
                    print("pass endSim......: non-control")
                    pass
                
                # Open, read and re-write contents to another file (in public folder)
                with open("temp_display.cur") as f:
                    with open("temp_display_.cur", "w") as f1:
                        for line in f:
                            if "error" not in line:
                                f1.write(line)

                # Copy the file (in public folder) to another prepared folder
                shutil.copy("temp_display_.cur", '/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles')
                print("copy successfully: non-control")

                # Rename the file in new folder
                os.rename('/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles/temp_display_.cur', 
                          r'/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles/temp_display_' + str(kp) + '-' + str(ki) + '-' + str(td) + 's' + '.cur')
                print("rename successfully: non-control")

                # Delete the file in public folder
                os.unlink("temp_display_.cur")
                print("delete successfully: non-control")
                
                print("flag = " + str(flag) + "\nReady to break: non-control\n")
                pass
            if flag == 1:
                del(ram)
                del(case)
                pass
            
            
            if flag == 0:
                # Initialization
                comp_type = ['SYN']
                comp_name = ['g2']
                obs_name = ['Omega']
                errSum = 0.0
                t=500
                nominal_frequency = 1.0
                list_of_gens = ['g6', 'g7', 'g14', 'g15', 'g16']

                # Name of bus 12 for checking voltage
                busNames = ['g12']

                # PI Control
                for i in np.arange(start_time+1,t):
                    #print("i = " + str(i))
                    actual_frequency = ram.getObs(comp_type,comp_name, obs_name)[0] # g2
                    error = nominal_frequency - actual_frequency
                    if abs(error)<0.00001:
                        error = 0.0
                    #print("error = " + str(error))

                    errSum += error * 1.0
                    #print("errSum = " + str(errSum))
                    output = float(kp) * float(error) + float(ki) * float(errSum)
                    if abs(output)<0.00001:
                        output = 0.0
                    # print("output = " + str(output))


                    # loop to send measurements to generators g6, g7, g14, g15, g16
                    for gen in list_of_gens:
                        command = 'CHGPRM TOR ' + gen + ' Tm0 ' + str(output/5.0) + ' 0'
                        #print(str(ram.getSimTime()+0.01)+' '+command)
                        td = float(td)
                        ram.addDisturb(ram.getSimTime() + td, command)

                    # Catch errors (voltages out of bound)
                    try: 
                        ram.contSim(i) # be parallel under the for loop (for gen in list_of_gens).
                    except:
                        print("RAMSES error => break....., ready to kill gnuplot")
                        break

                # Kill gnuplot
                os.system("TASKKILL /F /IM gnuplot.exe /T")
                print("kill successfully")

                # End simulation and exit
                try:
                    ram.endSim()
                    print("end simulation and exit successfully")
                except:
                    print("pass endSim......")
                    pass
                
                # Open, read and re-write contents to another file (in public folder)
                with open("temp_display.cur") as f:
                    with open("temp_display_.cur", "w") as f1:
                        for line in f:
                            if "error" not in line:
                                f1.write(line)

                # Copy the file (in public folder) to another prepared folder
                shutil.copy("temp_display_.cur", '/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles')
                print("copy successfully")

                td = "{0:.2f}".format(round(td,2))

                # Rename the file in new folder
                os.rename('/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles/temp_display_.cur', 
                          r'/Users/el17jg/Desktop/GitHub/Nordic-test-system/curfiles/temp_display_' + str(kp) + '-' + str(ki) + '-' + str(td) + 's' + '.cur')
                print("rename successfully")

                # Delete the file in public folder
                os.unlink("temp_display_.cur")
                print("delete successfully\n")
                
                del(ram)
                del(case)

kp = 5.60
ki = 3.60
td = 0.01
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 5.60
ki = 4.10
td = 0.01
kill successfully
end simulation and exit successfully
copy successfully
rename successfully
delete successfully

kp = 5.60
ki = 4.60
td = 0.01
RAMSES error => break....., ready to kill gnuplot
kill successfully
pass endSim......
copy successfully
rename successfully
delete successfully

kp = 5.60
ki = 5.10
td = 0.01
IMPORTANT TIPS: system will be quited and the file will be created WITHOUT control........
kill successfully: non-control
pass endSim......: non-control
copy successfully: non-control
rename successfully: non-control
delete successfully: non-control
flag = 1
Ready to break: non-control

kp = 5.60
ki = 5.60
td = 0.01
IMPORTANT TIPS: system will be quited and the file will be created WITHOUT control........
kill successfully: non-control
pass endSim......: non-control
copy successfully: non-control
renam